#### Capstone Project - Final Assigment

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Ratings](#ratings)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

##### In this final Capstone Projet we are going to  work with a group of investors that are trying to buy or invest on a Coffee Place in Paris. We gonna collect the information about the best coffe places in Paris and Try to se where is the best place to invest the Money.

##### We are going to use Foursquare for trying to get the best coffees around Center Paris, then load the Data, clean it, share it on a Map and then try to make some clusters.

##### First we gonna import some Libraries that we gonna use

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


## Data <a name="data"></a>

We gonna use the DATA extracted from Foursquare

* number of existing coffees in the neighborhood
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* Ratings of the Coffes
* Make a map with all the coffees and try to understand where is the best location


##### Define mi Credentials and Secret in Foursquare so we can collect the Data from Paris

In [84]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


### Search coordinate for Paris

In [3]:
address = 'Paris'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

48.8566969 2.3514616


#####  Search for Coffee venue category (Here we can see all the coffee places that are in the middle of Paris with a Radius of 2000)

In [4]:
search_query = 'Coffee'
radius = 2000
print(search_query + ' .... OK!')

Coffee .... OK!


##### Define URL 

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&ll=48.8566969,2.3514616&v=20180604&query=Coffee&radius=2000&limit=30'

##### Get Results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea053e429ce6a00285cf3d2'},
 'response': {'venues': [{'id': '52b5c225498eeda0297a855a',
    'name': 'Coffee Crêpes',
    'location': {'address': '24 quai du Louvre',
     'lat': 48.85884059518823,
     'lng': 2.3408023161365143,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.85884059518823,
       'lng': 2.3408023161365143}],
     'distance': 816,
     'postalCode': '75001',
     'cc': 'FR',
     'city': 'Paris',
     'state': 'Île-de-France',
     'country': 'France',
     'formattedAddress': ['24 quai du Louvre', '75001 Paris', 'France']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '478661351'},
    'referralId': 'v-1587565963',
    'hasPerk': False},
   {'id': '4b08f6

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.shape

(30, 19)

##### With this information we can see that we have 30 coffees that are in the Location we are searching. WE gona Filtered and try to get the names of the Coffees

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Coffee Crêpes,Coffee Shop,24 quai du Louvre,FR,Paris,France,NaN,816,"[24 quai du Louvre, 75001 Paris, France]","[{'label': 'display', 'lat': 48.85884059518823...",48.858841,2.340802,NaN,75001,Île-de-France,52b5c225498eeda0297a855a
1,Coffee Parisien,American Restaurant,4 rue Princesse,FR,Paris,France,NaN,1320,"[4 rue Princesse, 75006 Paris, France]","[{'label': 'display', 'lat': 48.85260711376305...",48.852607,2.334536,NaN,75006,Île-de-France,4b08f601f964a520981323e3
2,Starbucks Coffee - kiosque Gare de Est,Café,rue Du 8 Mai 1945,FR,Paris,France,NaN,2190,"[rue Du 8 Mai 1945, 75010 Paris, France]","[{'label': 'display', 'lat': 48.875763, 'lng':...",48.875763,2.358857,NaN,75010,Île-de-France,5d5a6e02b75d3c000930972a
3,Super Wild Coffee Gare de Est,Café,rue Du 8 Mai 1945,FR,Paris,France,NaN,2190,"[rue Du 8 Mai 1945, 75010 Paris, France]","[{'label': 'display', 'lat': 48.875763, 'lng':...",48.875763,2.358857,NaN,75010,Île-de-France,5bf955c200b068002dd619da
4,Thalone's Coffee,Café,NaN,FR,NaN,France,NaN,360,[France],"[{'label': 'display', 'lat': 48.85934789009945...",48.859348,2.348626,NaN,NaN,NaN,51b4aaaf498e8a8358aed7ec
5,Housey Coffee,None,NaN,FR,NaN,France,NaN,415,[France],"[{'label': 'display', 'lat': 48.8532581208, 'l...",48.853258,2.353684,NaN,NaN,NaN,5242d9d311d265868a6112d4
6,Motors Coffee,Café,7 rue des halles,FR,Paris,France,NaN,433,"[7 rue des halles, 75001 Paris, France]","[{'label': 'display', 'lat': 48.85931168681688...",48.859312,2.347072,NaN,75001,Île-de-France,59ce3e48aa6c9536b048846b
7,Costa Coffee,Coffee Shop,38 rue Rambuteau,FR,Paris,France,NaN,630,"[38 rue Rambuteau, 75003 Paris, France]","[{'label': 'display', 'lat': 48.86168533407975...",48.861685,2.347394,NaN,75003,Île-de-France,568d0d7c38fa97744916fa25
8,Coffee Spoune,Coffee Shop,36 rue Saint-Sébastien,FR,Paris,France,NaN,1507,"[36 rue Saint-Sébastien, 75011 Paris, France]","[{'label': 'display', 'lat': 48.86158015817987...",48.861580,2.370659,NaN,75011,Île-de-France,55b399d7498edddae2e9d040
9,Coffee Break,Café,NaN,FR,Paris,France,NaN,1010,"[Paris, France]","[{'label': 'display', 'lat': 48.86307054856677...",48.863071,2.341631,NaN,NaN,Île-de-France,501ba244e4b09166759c1e39


##### Let's Visuzalize the Coffees

In [9]:
dataframe_filtered.name

0                              Coffee Crêpes
1                            Coffee Parisien
2     Starbucks Coffee - kiosque Gare de Est
3              Super Wild Coffee Gare de Est
4                           Thalone's Coffee
5                              Housey Coffee
6                              Motors Coffee
7                               Costa Coffee
8                              Coffee Spoune
9                               Coffee Break
10                 Gagao Organic Coffee-Shop
11                                Sun Coffee
12                              Araku Coffee
13                           Savannah Coffee
14                               Coffee Shop
15                               Coffee Club
16                                Lux Coffee
17                Résidence Kann Coffee Shop
18                           Oni Coffee Shop
19                         Grain Coffee Shop
20                            Fashion Coffee
21                       Siva's Lunch Coffee
22        

##### Now we gonna represent all the Coffees in a Map

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Paris',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##### We can see that there are a lot of Coffees near the Channel La Seine. This is a place that is good for tourist and also for local people

##### We gonna explore some of the Coffes thar are near La Seine (our Point) and try to see their ratings: We gonna use Starbucks, Thalone's Coffe, Hoursey Coffee, Motors Coffee, Gagao Roganic Coffe, Costa Coffee, Coffee Crepes and Savanah Coffe wich are the ones that we handle to ass for investing or buying. We are not going to use Coffe Break and Sun Coffee. The owners are not open to sell.

In [11]:
CoffesID = dataframe_filtered[['name','distance','id']]

In [12]:
CoffesID.sort_values('distance').head(10)

,name,distance,id
28,Starbucks,252,4d9da49ec97a236afb68c199
4,Thalone's Coffee,360,51b4aaaf498e8a8358aed7ec
5,Housey Coffee,415,5242d9d311d265868a6112d4
6,Motors Coffee,433,59ce3e48aa6c9536b048846b
10,Gagao Organic Coffee-Shop,618,5cd57eba6bdee6002c1e4e0d
7,Costa Coffee,630,568d0d7c38fa97744916fa25
0,Coffee Crêpes,816,52b5c225498eeda0297a855a
13,Savannah Coffee,884,5288ba9911d2586d7c8a4d7e
9,Coffee Break,1010,501ba244e4b09166759c1e39
11,Sun Coffee,1017,582dbe1a0b057e44daae70ec


## Ratings <a name="ratings"></a>

#### Starbucks

In [13]:
venue_id = '4d9da49ec97a236afb68c199'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4d9da49ec97a236afb68c199?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [14]:
result = requests.get(url).json()

##### Get the venue's overall rating

In [15]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


#### Thalones Coffee

In [16]:
venue_id2 = '51b4aaaf498e8a8358aed7ec' # Thalones Coffee
url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id2, CLIENT_ID, CLIENT_SECRET, VERSION)
url2

'https://api.foursquare.com/v2/venues/51b4aaaf498e8a8358aed7ec?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [17]:
result2 = requests.get(url2).json()

In [18]:
try:
    print(result2['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


#### Housey Coffee

In [19]:
venue_id3 = '5242d9d311d265868a6112d4'
url3 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id3, CLIENT_ID, CLIENT_SECRET, VERSION)
url3

'https://api.foursquare.com/v2/venues/5242d9d311d265868a6112d4?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [20]:
result3 = requests.get(url3).json()

In [21]:
try:
    print(result3['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


#### Motors Coffee

In [22]:
venue_id4 = '59ce3e48aa6c9536b048846b'
url4 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id4, CLIENT_ID, CLIENT_SECRET, VERSION)
url4

'https://api.foursquare.com/v2/venues/59ce3e48aa6c9536b048846b?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [23]:
result4 = requests.get(url4).json()

In [24]:
try:
    print(result4['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


#### Gagao Organic Coffee-Shop

In [25]:
venue_id5 = '5cd57eba6bdee6002c1e4e0d'
url5 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id5, CLIENT_ID, CLIENT_SECRET, VERSION)
url5

'https://api.foursquare.com/v2/venues/5cd57eba6bdee6002c1e4e0d?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [26]:
result5 = requests.get(url5).json()

In [27]:
try:
    print(result5['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


#### Costa Coffee

In [28]:
venue_id6 = '568d0d7c38fa97744916fa25'
url6 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id6, CLIENT_ID, CLIENT_SECRET, VERSION)
url6

'https://api.foursquare.com/v2/venues/568d0d7c38fa97744916fa25?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [29]:
result6 = requests.get(url6).json()

In [30]:
try:
    print(result6['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


#### Coffee Crêpes

In [31]:
venue_id7 = '52b5c225498eeda0297a855a'
url7 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id7, CLIENT_ID, CLIENT_SECRET, VERSION)
url7

'https://api.foursquare.com/v2/venues/52b5c225498eeda0297a855a?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [32]:
result7 = requests.get(url7).json()

In [33]:
try:
    print(result7['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.7


#### Savannah Coffee

In [34]:
venue_id8 = '5288ba9911d2586d7c8a4d7e'
url8 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id8, CLIENT_ID, CLIENT_SECRET, VERSION)
url8

'https://api.foursquare.com/v2/venues/5288ba9911d2586d7c8a4d7e?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&v=20180604'

In [35]:
result8 = requests.get(url8).json()

In [36]:
try:
    print(result8['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


##### After searching Locations and doing Ratins we agree that Coffe Crepes is the best place for investing. We had a problem that most of the places had not been rated. But we have an oustading Rating of 8.7 in Crepes, better than Savanahh 7.4, Motor 7.3 and Starbucks 7.1

##### Coffe Crepes have a good score, so we should see how they work to get some tips

In [37]:
result7['response']['venue']['tips']['count']

93

In [67]:
## Mascalzone Tips
limit = 15
url7 = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id7, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

result = requests.get(url7).json()
result

{'meta': {'code': 200, 'requestId': '5ea05737bae9a2001b530ee6'},
 'response': {'tips': {'count': 94,
   'items': [{'id': '59b68ec18134882faa151b8b',
     'createdAt': 1505136321,
     'text': "Gaufre révolution c'est son nom, belle gaufre dorée banane chantilly et glace vanille avec chocolat chaud !!! Wwhhaoouu !!!! Super bon et j'ai bu un chocolat chaud, Super mousseux !!!❤️❤️❤️",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/59b68ec18134882faa151b8b',
     'photo': {'id': '59b68ed6ea1e44219bc23cca',
      'createdAt': 1505136342,
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/450926356_OsDJAg8wKk0aM_27WwGaRj_ikVWbYmyEs_pQ6tDzDLg.jpg',
      'width': 1920,
      'height': 1440,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/450926356_OsDJAg8wKk0aM_27WwGaRj_ikVWbYmyEs_pQ6tDzDLg.jpg',


In [64]:
tips = result['response']['tips']['items']

tip = result['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

Display Tips

In [65]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Gaufre révolution c'est son nom, belle gaufre dorée banane chantilly et glace vanille avec chocolat chaud !!! Wwhhaoouu !!!! Super bon et j'ai bu un chocolat chaud, Super mousseux !!!❤️❤️❤️",23,0,59b68ec18134882faa151b8b,Jeanine,A,NaN,450926356


##### So we define that we are going to invest on Coffee Crepes, the Location is near where we want it and they have the best rating. Now, lets explore the locations and what we have near.

In [70]:
dflatlong = dataframe_filtered[['name','lat','lng']]

In [72]:
dflatlong.head(1)

,name,lat,lng
0,Coffee Crêpes,48.858841,2.340802


In [74]:
latitude = 48.858841
longitude = 2.340802

In [75]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=5JUO2XVTNTGAMQZ2BBW55NBR33CED2EXMDT2TLGNKVWM0EOE&client_secret=EXOFUWQRDSQSWD0PQ2XESIJLDRYJAQS4SAT30QZWELDHGIDV&ll=48.858841,2.340802&v=20180604&radius=2000&limit=30'

In [76]:
import requests

In [80]:
results = requests.get(url).json()
'There are {} popular spots around Coffe Crepes .'.format(len(results['response']['groups'][0]['items']))

'There are 30 popular spots around Coffe Crepes .'

In [79]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4f979c8ae4b05465dae0714f',
  'name': 'Place du Louvre',
  'location': {'address': 'Place du Louvre',
   'lat': 48.859841,
   'lng': 2.340822,
   'labeledLatLngs': [{'label': 'display', 'lat': 48.859841, 'lng': 2.340822}],
   'distance': 111,
   'postalCode': '75001',
   'cc': 'FR',
   'city': 'Paris',
   'state': 'Île-de-France',
   'country': 'France',
   'formattedAddress': ['Place du Louvre', '75001 Paris', 'France']},
  'categories': [{'id': '4bf58dd8d48988d164941735',
    'name': 'Plaza',
    'pluralName': 'Plazas',
    'shortName': 'Plaza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4f979c8ae4b05465dae0714f-0'}

In [81]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Place du Louvre,Plaza,Place du Louvre,FR,Paris,France,NaN,111,"[Place du Louvre, 75001 Paris, France]","[{'label': 'display', 'lat': 48.859841, 'lng': 2.340822}]",48.859841,2.340822,NaN,75001,Île-de-France,4f979c8ae4b05465dae0714f
1,Cour Carrée du Louvre,Pedestrian Plaza,Rue de Rivoli,FR,Paris,France,Place du Louvre,236,"[Rue de Rivoli (Place du Louvre), 75001 Paris, France]","[{'label': 'display', 'lat': 48.8603598795578, 'lng': 2.3385429382324214}]",48.860360,2.338543,NaN,75001,Île-de-France,4c079d740ed3c928b6be797d
2,Pont des Arts,Bridge,Pont des Arts,FR,Paris,France,NaN,233,"[Pont des Arts, 75001 Paris, France]","[{'label': 'display', 'lat': 48.85856468168776, 'lng': 2.3376350104808807}]",48.858565,2.337635,NaN,75001,Île-de-France,4adcda09f964a520ff3321e3
3,La Vénus de Milo (Vénus de Milo),Exhibit,Musée du Louvre,FR,Paris,France,"Aile Sully, rdc, salle 346",288,"[Musée du Louvre (Aile Sully, rdc, salle 346), 75001 Paris, France]","[{'label': 'display', 'lat': 48.859943428504316, 'lng': 2.3372340202331543}]",48.859943,2.337234,"Louvre, Paris",75001,Île-de-France,5864efb745c3ed1e7d88e96d
4,Place Dauphine,Plaza,Place Dauphine,FR,Paris,France,NaN,270,"[Place Dauphine, 75001 Paris, France]","[{'label': 'display', 'lat': 48.85663993129474, 'lng': 2.342362403869629}]",48.856640,2.342362,NaN,75001,Île-de-France,4adcda09f964a520eb3321e3
5,Chez Nous,Wine Bar,10 rue Dauphine,FR,Paris,France,NaN,338,"[10 rue Dauphine, 75006 Paris, France]","[{'label': 'display', 'lat': 48.855823217287835, 'lng': 2.340271085661046}]",48.855823,2.340271,NaN,75006,Île-de-France,53568d3e498e92e5a3a56f36
6,Musée du Louvre,Art Museum,Rue de Rivoli,FR,Paris,France,Place du Carrousel,389,"[Rue de Rivoli (Place du Carrousel), 75001 Paris, France]","[{'label': 'display', 'lat': 48.86084691113991, 'lng': 2.336440086364746}]",48.860847,2.336440,Le Louvre,75001,Île-de-France,4adcda10f964a520af3521e3
7,Guy Savoy,French Restaurant,11 quai de Conti,FR,Paris,France,Monnaie de Paris,244,"[11 quai de Conti (Monnaie de Paris), 75006 Paris, France]","[{'label': 'display', 'lat': 48.857085019941806, 'lng': 2.3388068135574938}]",48.857085,2.338807,NaN,75006,Île-de-France,5540b49d498e84ce71f6b0ce
8,Monnaie de Paris,Museum,11 quai de Conti,FR,Paris,France,NaN,257,"[11 quai de Conti, 75006 Paris, France]","[{'label': 'display', 'lat': 48.85683338028305, 'lng': 2.3390582159751068}]",48.856833,2.339058,NaN,75006,Île-de-France,4adcda0ef964a520353521e3
9,Ze Kitchen Galerie,Restaurant,4 rue des Grands Augustins,FR,Paris,France,NaN,417,"[4 rue des Grands Augustins, 75006 Paris, France]","[{'label': 'display', 'lat': 48.85510110130317, 'lng': 2.341297271887015}]",48.855101,2.341297,NaN,75006,Île-de-France,4b4b6fe4f964a520a19b26e3


In [83]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## Conclusion <a name="conclusion"></a>

##### After doing this analysis, we can see that Coffee Crepes is the best place to invest. It is close to popular places like the Musée du Louvre and other historical and tourist places. It has an almost excellent rating in Foursquare after comparting to the other options that where offer to us for investing or buying. Also, it is close to our ideal spot in middle Paris.